In [1]:
%matplotlib inline
%config InlineBackend.close_figures=False 

import matplotlib.pyplot as plt
import numpy as np

import ipywidgets as widgets
from IPython.display import display,clear_output

In [2]:
from ANEMO import ANEMO
from ANEMO import read_edf

import os
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
sujet = ['AM','BMC','CS','DC','FM','IP','LB','OP','RS','SR','TN','YK']
time = ['2017-10-23_100057','2017-09-26_095637',
        '2017-10-03_134421','2017-09-27_161040',
        '2017-10-03_143803','2017-09-28_115250',
        '2017-09-20_151043','2017-10-26_121823',
        '2017-11-08_094717','2017-11-16_153313',
        '2017-11-08_150410','2017-11-17_172706']

In [4]:
x = 0

In [5]:
file = os.path.join('data', 'enregistrement_' + sujet[x] + '_' + time[x] + '.pkl')
with open(file, 'rb') as fichier :
    param_exp = pickle.load(fichier, encoding='latin1')


resultats = os.path.join('data', 'enregistrement_' + sujet[x] + '_' + time[x] + '.asc')
data = read_edf(resultats, 'TRIALID')

list_fct = {'fct_velocity':'velocity', 'fct_position':'position', 'fct_saccade':'position'}
Full_param_fit = {}
for fct in list_fct.keys() :
    try :
        file = os.path.join('parametre/param_Fit_'+ sujet[x] +'_' + fct + '_2_step_False_whitening.pkl')
        print(file)
        with open(file, 'rb') as fichier :
            Full_param_fit[fct] = pickle.load(fichier, encoding='latin1')
    except :
        Full_param_fit[fct] = None

parametre/param_Fit_AM_fct_saccade_2_step_False_whitening.pkl
parametre/param_Fit_AM_fct_velocity_2_step_False_whitening.pkl
parametre/param_Fit_AM_fct_position_2_step_False_whitening.pkl


In [6]:
Plot = ANEMO.Plot(param_exp)
Fit = ANEMO.Fit(param_exp)
ANEMO = ANEMO(param_exp)

In [7]:
list(list_fct.keys())[0]

'fct_saccade'

In [8]:
def test_fig(param_exp, data, list_fct, Full_param_fit) :
    
    N_trials, N_blocks = param_exp['N_trials'], param_exp['N_blocks']
    list_param_enre=['start_anti', 'v_anti', 'latence', 'tau', 'maxi']
        
        
    
    list_ok, list_bad = {}, {}
    for fct in list_fct.keys() :
        list_ok[fct], list_bad[fct] = [], []
        for block in range(N_blocks) :
            list_ok[fct].append([])
            list_bad[fct].append([])
    
    trial, block = 0, 0
    fct= list(list_fct.keys())[2]
    param_fit = Full_param_fit[fct]
    fitted_data = list_fct[fct]

    plt.ioff()
    fig, ax = plt.subplots(1, 1, figsize=(12, (12*(1/2)/1.6180)))

    h = '%spx'%((fig.get_figheight()+1)*fig.get_dpi())
    w = '%spx'%((fig.get_figwidth()+1)*fig.get_dpi())
    
    out = widgets.Output(layout=widgets.Layout(grid_area='out'))
    
    button = {}
    for b, style in zip(['Previews', 'Next', 'OK', 'Bad'], ['', '', 'success', 'danger']) :
        button[b] = widgets.Button(description=b, button_style=style,
                                  layout=widgets.Layout(grid_area=b, margin='auto 0%', width='90%'))
    
    _fct_fit = widgets.Dropdown(value=fct, options=list_fct.keys(),
                                description='fct_fit', disabled=False,
                                layout=widgets.Layout(grid_area='fct_fit', margin='auto 0%', width='90%'))
    
    _trial = widgets.IntSlider(value=trial, min=0, max=N_trials-1, step=1, description='Trial', readout=True,
                                 layout=widgets.Layout(grid_area='trial', margin='auto 0%', width='90%'))
    
    
    
    _block = widgets.Dropdown(value='%s'%block, options=['0', '1', '2'], description='Block', disabled=False,
                                 layout=widgets.Layout(grid_area='block', margin='auto 0%', width='90%'))
    
    grid = widgets.GridBox(children=[button['OK'], button['Bad'], _fct_fit,
                                     button['Previews'], _trial, _block, out, button['Next']],
                           layout=widgets.Layout(grid_template_columns='10% 10% 50% 10% 10% 10%',
                                                 grid_template_areas=''' "OK Bad . fct_fit fct_fit ."
                                                                        "out out out out out out"
                                                                       "Previews . trial block . Next" '''))
    
    display(grid)
    

    
    def fig(title, c, out) :
        nonlocal trial, block, ax, fct, fitted_data, param_fit
        param_f = Plot.show_fit(ax, trial, block, title, c, out, data=data,
                                param_fit=param_fit, equation=fct, fitted_data=fitted_data,
                                list_param_enre=list_param_enre)
    
    def check_fct(b) :
        nonlocal fct, fitted_data, param_fit
        
        fct = _fct_fit.value
        fitted_data = list_fct[fct]
        param_fit = Full_param_fit[fct]
        
        c, title = check_list()
        fig(title, c, out)
    
    def check_list() :
        nonlocal trial, block, fct, list_bad, list_ok
        
        if trial in list_bad[fct][block] : c='darkred' ; title=' -- Bad'
        elif trial in list_ok[fct][block] : c='darkgreen' ; title=' -- OK'
        else : c='k' ; title=''
        return c, title
    
    def check_trial(b) :
        nonlocal trial
        trial =  _trial.value
        c, title = check_list()
        fig(title, c, out)
    
    def check_block(b) :
        nonlocal block
        block = int(_block.value)
        c, title = check_list()
        fig(title, c, out)
    
    def Next(b):
        nonlocal trial, block
        trial = trial+1
        if trial >= N_trials : trial = 0 ; block = block+1
        if block >= N_blocks : trial = N_trials-1 ; block = N_blocks-1
        _trial.value, _block.value = trial, '%s'%block

    def Previews(b):
        nonlocal trial, block
        trial = trial-1
        if trial < 0 : trial = N_trials-1 ; block = block-1
        if block < 0 : trial = 0 ; block = 0
        _trial.value, _block.value = trial, '%s'%block
        
    def Bad(b):
        nonlocal trial, block, list_bad, list_ok
        if trial not in list_bad[fct][block] :
            list_bad[fct][block].append(trial) ; list_bad[fct][block].sort()
        if trial in list_ok[fct][block] :
            del list_ok[fct][block][list_ok[fct][block].index(trial)] ; list_ok[fct][block].sort()
        fig(' -- Bad', 'r', out)

    def no_Bad(b):
        nonlocal trial, block, list_bad, list_ok
        if trial not in list_ok[fct][block] :
            list_ok[fct][block].append(trial) ; list_ok[fct][block].sort()
        if trial in list_bad[fct][block] :
            del list_bad[fct][block][list_bad[fct][block].index(trial)] ; list_bad[fct][block].sort()
        fig(' -- OK', 'g', out)
    
    button['Previews'].on_click(Previews)
    button['Bad'].on_click(Bad)
    button['OK'].on_click(no_Bad)
    button['Next'].on_click(Next)
    
    _trial.observe(check_trial, names='value')
    _block.observe(check_block, names='value')
    _fct_fit.observe(check_fct, names='value')

    check_trial(None)
    return list_bad, list_ok

In [9]:
list_bad, list_ok = test_fig(param_exp, data, list_fct, Full_param_fit)

GridBox(children=(Button(button_style='success', description='OK', layout=Layout(grid_area='OK', margin='auto …

In [14]:
print(list_bad)
print(list_ok)

{'fct_saccade': [[], [113], []], 'fct_velocity': [[], [], []], 'fct_position': [[198, 199], [113], [16, 57, 199]]}
{'fct_saccade': [[113], [], []], 'fct_velocity': [[], [5], []], 'fct_position': [[191, 194], [5], [183]]}
